In [18]:
from keras.optimizers import SGD
from keras.layers import Input, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
import keras.backend as K 
from sklearn.metrics import log_loss
from custom_layers.scale_layer import Scale
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split   
from keras.layers import Concatenate






ImportError: cannot import name 'img_to_array' from 'keras.preprocessing.image' (C:\Users\Spencer\anaconda3\envs\mlenv\lib\site-packages\keras\preprocessing\image.py)

In [ ]:
#pip install opencv-python

In [12]:

def densenet121_model(img_rows, img_cols, color_type=1, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.5, dropout_rate=0.0, weight_decay=1e-4, num_classes=None):
    '''
    DenseNet 121 Model for Keras

    Model Schema is based on 
    https://github.com/flyyufelix/DenseNet-Keras

    # Returns
        A Keras model instance.
    '''

    # Handle Dimension Ordering for different backends
    global concat_axis
    img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
    concat_axis = 3


    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6,12,24,16] # For DenseNet-121

    # Initial convolution
    x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(img_input)
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, dropout_rate=dropout_rate)
        nb_filter = int(nb_filter)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate)

    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis )(x)
    x = Activation('relu')(x)

    x_fc = GlobalAveragePooling2D()(x)
    x_fc = Dense(1000)(x_fc)
    x_fc = Activation('softmax')(x_fc)

    model = Model(img_input, x_fc)

    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = GlobalAveragePooling2D()(x)
    x_newfc = Dense(num_classes)(x_newfc)
    x_newfc = Activation('softmax')(x_newfc)

    model = Model(img_input, x_newfc)

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def conv_block(x, stage, branch, nb_filter, dropout_rate=None):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout '''


    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(inter_channel, 1, 1, bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D((1, 1))(x)
    x = Convolution2D(nb_filter, 3, 3, bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, dropout_rate=None):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout  '''


    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(nb_filter), 1, 1, bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    concat_feat = x

    for i in range(nb_layers):
        branch = i + 1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate)
        concat_feat = Concatenate(axis=concat_axis)([concat_feat, x])
        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [13]:
import os
import random
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


def load_data(train_directory, test_directory, target_size=(128, 128)):
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    random.seed(42)

    # Load training data
    for class_dir in os.listdir(train_directory):
        image_paths = sorted(list(os.listdir(os.path.join(train_directory, class_dir))))
        for img_path in image_paths:
            img_full_path = os.path.join(train_directory, class_dir, img_path)
            if os.path.isfile(img_full_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                print(f"Loading training image: {img_full_path}")
                # Load and preprocess the image
                image = cv2.imread(img_full_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                image = cv2.resize(image, target_size)
                train_data.append(image)
                train_labels.append(class_dir)

    # Load testing data
    for class_dir in os.listdir(test_directory):
        image_paths = sorted(list(os.listdir(os.path.join(test_directory, class_dir))))
        for img_path in image_paths:
            img_full_path = os.path.join(test_directory, class_dir, img_path)
            if os.path.isfile(img_full_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                print(f"Loading testing image: {img_full_path}")
                # Load and preprocess the image
                image = cv2.imread(img_full_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                image = cv2.resize(image, target_size)
                test_data.append(image)
                test_labels.append(class_dir)

    # Convert data and labels to NumPy arrays
    train_data = np.array(train_data, dtype="float32") / 255.0
    train_labels = np.array(train_labels)
    test_data = np.array(test_data, dtype="float32") / 255.0
    test_labels = np.array(test_labels)

    # One-hot encode the labels
    mlb = LabelBinarizer()
    train_labels = mlb.fit_transform(train_labels)
    test_labels = mlb.transform(test_labels)

    print("Data loaded successfully.")
    return train_data, train_labels, test_data, test_labels

# Define the directories for train and test data
train_directory = 'C:\\Users\\Spencer\\Desktop\\Physics\\Y3S2 Colgate\\COSC410L\\Final Project\\Train\\church'
test_directory = 'C:\\Users\\Spencer\\Desktop\\Physics\\Y3S2 Colgate\\COSC410L\\Final Project\\Test'

# Load the data
x_train, y_train, x_test, y_test = load_data(train_directory, test_directory)


Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00001.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00002.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00003.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00004.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00005.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00006.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00007.png
Loading training image: C:\Users\Spencer\Desktop\Physics\Y3S2 Colgate\COSC410L\Final Project\Train\church\images\00008.png
Loading training

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\Spencer\\Desktop\\Physics\\Y3S2 Colgate\\COSC410L\\Final Project\\Train\\church\\LICENSE.txt'